In [1]:
# Import necessary packages
import pandas as pd
from pymongo import MongoClient
import time
import psutil
import cudf
from extract_load_functions_mongodb import extraction, loading, performance

#set export location
exportLocation = r'/home/jeff/'

#set number of test runs to prefrom
iterations = 31

# Connect to MongoDb
myclient = MongoClient("mongodb://localhost:27017/")
db = myclient.ds7330 
item = db.item
order_line = db.order_line

etlTimerStart = time.perf_counter()    
# MEDIUM EXTRACTION #
###################################################################################################################
### Pull item table and capture CPU, RAM and elapsed time to perform operations

extractionTimerStart = time.perf_counter()
### Run 30 iterations to collect a df of performance metrics
### light extraction performance metrics
column_names = ["base_CPU","base_RAM","CPU_utilization", "RAM_utilization", "CPU_d","RAM_d","elapsed_time"]
medExtractionPrfm = pd.DataFrame(columns = column_names)

### base metrics
print("STARTING EXTRACTION...")
print("RUNNING...")

### Run n iterations to collect a df of performance metrics
for sampleNo in range(iterations):
    
    time.sleep(5)
    #collect base settings
    baseCPU = psutil.cpu_percent()
    baseRAM = psutil.virtual_memory().percent
    
    #Initiate timer for query
    start = time.perf_counter()

    #insert data into dataFrame
    itemdf = pd.DataFrame(list(item.find()))
    order_linedf = pd.DataFrame(list(order_line.find()))
    sampleCPU = psutil.cpu_percent()
    sampleRAM = psutil.virtual_memory().percent
    
    #Stop timer  
    stop = time.perf_counter()

    #load df with performance metrics    
    medExtractionPrfm = medExtractionPrfm.append(pd.DataFrame({'base_CPU': baseCPU,
                                                         'base_RAM': baseRAM,
                                                         'CPU_utilization': sampleCPU,
                                                         'RAM_utilization': sampleRAM,
                                                         'CPU_d': sampleCPU-baseCPU,
                                                         'RAM_d': sampleRAM - baseRAM,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)
extractionTimerEnd= time.perf_counter()   

print("Data frame loading complete.\n")
performance(medExtractionPrfm, iterations, extractionTimerStart,extractionTimerEnd)

#Export DF to csv
medExtractionPrfm.to_csv (r'/home/jeff/medExtractionPrfm_MongoDB.csv', index = False, header=True)

#### MEDIUM TRANSFORMATION ####
###################################################################################################################
# Preform medium transformation workload.  In this case, join orderline and item tables, identify all item descriptions with "Blue" in them 
# and change them to "Navy"

#Cast datatypes to objects
order_linedf[['Line', 'eaches_qty']] = order_linedf[['Line', 'eaches_qty']].apply(pd.to_numeric) 
order_linedf[['Order', 'orderedItem','Ponum']] = order_linedf[['Order', 'orderedItem','Ponum']].astype(str) 

itemdf[['selling_price']] = itemdf[['selling_price']].apply(pd.to_numeric)
itemdf[['orderedItem']] = itemdf[['orderedItem']].astype(str)

column_names = ["base_CPU","base_RAM","CPU_utilization", "RAM_utilization", "CPU_d","RAM_d","elapsed_time"]
medTransPrfm = pd.DataFrame(columns = column_names)
#lightTrans.item_desc.str.contains("^Blue")

print("STARTING TRANSFORMATION...")
print("Base CPU utilization: ", psutil.cpu_percent())
print("Base RAM utilization: ", psutil.virtual_memory().percent)
print("RUNNING...")

transTimerStart = time.perf_counter()
#Run 30 iterations to collect transformation df of performance metrics

for sampleNoTransform in range(iterations):
    time.sleep(5)
    #collect base settings
    baseCPU = psutil.cpu_percent()
    baseRAM = psutil.virtual_memory().percent
    
    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    itemtrans = itemdf.drop(index=itemdf.index[[-1]])
    order_linetrans = order_linedf.drop(index=order_linedf.index[[-1]])
    
    itemtrans =itemdf.drop(itemdf.columns[[0]], axis = 1) 
    order_linetrans = order_linedf.drop(order_linedf.columns[[0]], axis = 1) 
    
    medium = order_linetrans.merge(right=itemtrans, on="orderedItem")
    
    medium.item_desc = medium.item_desc.str.replace('Blue', 'Navy', regex=True)
    medium['extended_price'] = medium.selling_price * medium.eaches_qty
    sampleCPU = psutil.cpu_percent()
    sampleRAM = psutil.virtual_memory().percent
    
    #Stop timer  
    stop = time.perf_counter()
    
    
    medTransPrfm = medTransPrfm.append(pd.DataFrame({'base_CPU': baseCPU,
                                                         'base_RAM': baseRAM,
                                                         'CPU_utilization': sampleCPU,
                                                         'RAM_utilization':  sampleRAM,
                                                         'CPU_d': sampleCPU-baseCPU,
                                                         'RAM_d': sampleRAM - baseRAM,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)
transTimerEnd= time.perf_counter() 

print("Pandas transformation metrics captured.\n")
performance(medTransPrfm,iterations,transTimerStart,transTimerEnd)

# Export Light Transformation Performance to local
medTransPrfm.to_csv (r'/home/jeff/medTransPrfm_mongoDB.csv', index = False, header=True)


#### cuDF TRANSFORMATION ####
###################################################################################################################

column_names = ["base_CPU","base_RAM","CPU_utilization", "RAM_utilization", "CPU_d","RAM_d","elapsed_time"]
medTransPrfmCU = pd.DataFrame(columns = column_names)

print("Starting cuDF TRANSFORMATION...")
print("RUNNING...")

#Run 30 iterations to collect transformation df of performance metrics
transCuTimerStart = time.perf_counter()
for sampleNoTransform in range(iterations):
    time.sleep(5)
    #collect base settings
    baseCPU = psutil.cpu_percent()
    baseRAM = psutil.virtual_memory().percent
    
    ## Transform pandas df to cuDF
    itemCU = cudf.DataFrame.from_pandas(itemtrans)
    order_lineCU = cudf.DataFrame.from_pandas(order_linetrans)
    
    # Start Timer and progress tracker
    start = time.perf_counter()
    
    #Working Code
    joined_dataCU = order_lineCU.merge(right=itemCU, on="orderedItem")
    
    joined_dataCU.item_desc = joined_dataCU.item_desc.str.replace('Blue', 'Navy', regex=True)
    joined_dataCU['extended_price'] = joined_dataCU.selling_price * joined_dataCU.eaches_qty
    sampleCPU = psutil.cpu_percent()
    sampleRAM = psutil.virtual_memory().percent
    
    #Stop timer  
    stop = time.perf_counter()
    
    medTransPrfmCU = medTransPrfmCU.append(pd.DataFrame({'base_CPU': baseCPU,
                                                         'base_RAM': baseRAM,
                                                         'CPU_utilization': sampleCPU,
                                                         'RAM_utilization':  sampleRAM,
                                                         'CPU_d': sampleCPU-baseCPU,
                                                         'RAM_d': sampleRAM - baseRAM,
                                                         'elapsed_time':stop - start},
                                                          index=[1]), ignore_index=True)
    #lightTrans = orderItemJoin
    time.sleep(2)
transCuTimerEnd = time.perf_counter() 

print("cuDF performance metrics captured loading complete.\n")
performance(medTransPrfmCU,iterations,transCuTimerStart,transCuTimerEnd)

# Export medium cuDF Transformation Performance to local
medTransPrfmCU.to_csv (r'/home/jeff/mediumTransPrfmCU_mongoDB.csv', index = False, header=True)

#### Load data ####
###################################################################################################################
loading(myclient,'medium', medium, iterations, exportLocation, "mediumMongoDBLoad.csv")

etlTimerEnd = time.perf_counter() 
print('ETL is complete')
print('Elapsed ETL time is: ', (etlTimerEnd-etlTimerStart)/60, ' minutes')

STARTING EXTRACTION...
RUNNING...
Data frame loading complete.

Iterations performed:  31
TOTAL Process time:  201.1200917170063 s
Average iteration time:  1.4819002530641756 s
Average BASE CPU:  1.709677419354839
Average BASE RAM:  29.232258064516113
Average CPU Performance  4.129032258064516
Average RAM Performance  0.035483870967741984
Average CPU utilization:  5.838709677419353
Average RAM utilization:  29.267741935483855 


STARTING TRANSFORMATION...
Base CPU utilization:  6.5
Base RAM utilization:  29.2
RUNNING...
Pandas transformation metrics captured.

Iterations performed:  31
TOTAL Process time:  162.13106010000047 s
Average iteration time:  0.22488153903208102 s
Average BASE CPU:  1.4129032258064513
Average BASE RAM:  29.412903225806446
Average CPU Performance  4.170967741935484
Average RAM Performance  0.006451612903225784
Average CPU utilization:  5.583870967741936
Average RAM utilization:  29.41935483870967 


Starting cuDF TRANSFORMATION...
RUNNING...
cuDF performance me